# Test out `topp.py`, `util.py`, `io.py`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import src.io as io
import src.topp as topp
import src.util as util
from src.communicate import CableRobot
plt.rc('axes', unicode_minus=False)  # Glyph 8722

In [ ]:
strokes = io.load_log('logs/G.txt')
for stroke in strokes:
    print(stroke.shape)

In [ ]:
# Plot strokes
strokes = util.clean_strokes(strokes)
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for t, xy in util.strokes2txs(strokes):
    xydot, xyddot = util.derivatives(t, xy)
    axes[0].plot(*xy.T)
    axes[1].plot(t, np.linalg.norm(xydot, axis=1))
    axes[2].plot(t, np.linalg.norm(xyddot, axis=1))
axes[0].axis('equal')
axes[0].set_title('xy')
axes[1].set_title('Speed vs time')
axes[2].set_title('||Acceleration|| vs time');

In [ ]:
# Do TOPP-RA
stroke = topp.Stroke(strokes[0], clean=True)
stroke.spline_interp()
stroke.retime(vmax=1, amax=5)

ts, xs, xds, xdds = stroke.sample_retimed(N=200)

In [ ]:
# Plot TOPP-RA results
fig, axes = plt.subplots(3, 2, figsize=(5, 6))
stroke.plot_xva(axes)
fig.tight_layout()

In [ ]:
# Create html animation
HTML(stroke.create_html_anim('test1.html'))

# Investigate the data format of `toppra.SplineInterpolator`
(to reverse-engineer)

In [ ]:
coeff = stroke.path.cspl.c
coeffv = stroke.path.cspld.c
breakpts = stroke.path.cspl.x
print(coeff.shape) # (degree, numsegments, xy)
print(breakpts.shape) # (numsegments+1)

In [ ]:
print(coeff.shape, coeffv.shape) # coeffv should have degree 1 less than coeff
print(coeff[:, 10, 0])
print(coeffv[:, 10, 0])
print(coeff[:, 10, 0] * np.arange(3, -1, -1)) # manually take the derivative to compare to coeffv

In [ ]:
# Now try to manually evaluate the piecewise polynomial!
t = np.array([6e-4, 0.2, 0.4, 0.6, 1])
expected = stroke.path(t)
print(expected)

# manually evaluate the spline
t2 = t.reshape(-1, 1)
cond = np.logical_and(t2 >= breakpts[:-1].reshape(1, -1), t2 < breakpts[1:].reshape(1, -1))
xy = []
for t_, col in zip(t, cond):
    coeff_ = coeff[:, col, :].squeeze()
    N = coeff_.shape[0] - 1
    t_ = t_ - breakpts[:-1][col]
    tpow = np.power(t_, np.arange(N, -1, -1)).reshape(1, -1)
    xy.append(tpow @ coeff_)
actual = np.array(xy).squeeze()
print(actual)
print(expected - actual)

In [ ]:
# Implemented the above code in `topp.manually_evaluate_spline`
# Test `topp.manually_evaluate_spline`
t = np.array([6e-4, 0.2, 0.4, 0.6, 1])
expected = stroke.path(t)
actual = topp.manually_evaluate_spline(t, stroke.path)
np.testing.assert_allclose(actual, expected, rtol=1e-15, atol=1e-15)

# Create data for c++ unit tests

In [ ]:
xc = [1, 2, 3, 4]
yc = [5, 6, 7, 8]
t = 0.1
xdc, ydc = np.polyder(xc), np.polyder(yc)
xddc, yddc = np.polyder(xdc), np.polyder(ydc)
for t in [0.05, 0.09999999]:
    print(f'EXPECT_XVA_EQUAL({t}, 1e-6,             // t, tol')
    print(f'                 {np.polyval(xc, t):.6f}, {np.polyval(yc, t):.6f},    // x')
    print(f'                 {np.polyval(xdc, t):.6f}, {np.polyval(ydc, t):.6f},    // v')
    print(f'                 {np.polyval(xddc, t):.6f}, {np.polyval(yddc, t):.6f});  // a')

In [ ]:
for i in range(4):
    b = breakpts[i+1]
    c = coeff[:, i, :].squeeze()
    row2str = lambda row: ', '.join(f'{n:.6f}' for n in row)
    print(f'spline.add_segment({b}, {{{{ {{{{ {row2str(c[:, 0])} }}}}, {{{{ {row2str(c[:, 1])} }}}} }}}});')

In [ ]:
for t in [0.005, 0.01, 0.015, breakpts[2], 0.035, 0.045]:
    print(f'EXPECT_XVA_EQUAL({t}, 1e-6,            // t, tol')
    print(f'                 {row2str(stroke.path.eval(t))},  // x')
    print(f'                 {row2str(stroke.path.evald(t))},  // v')
    print(f'                 {row2str(stroke.path.evaldd(t))});  // a')
t = breakpts[4]
print(f'EXPECT_XVA_EQUAL({0.05}, 1e-6,            // t, tol')
print(f'                 {row2str(stroke.path.eval(t))},  // x')
print(f'                 {row2str(stroke.path.evald(t))},  // v')
print(f'                 {row2str(stroke.path.evaldd(t))});  // a')

# Re-scale and Send over serial to robot

In [ ]:
PORT = '/dev/tty.usbmodem100994303'

In [ ]:
W, H = 2.92, 2.32
center = np.mean([np.max(stroke.x, axis=0), np.min(stroke.x, axis=0)], axis=0)
newx = stroke.x - center + [W / 2, H / 2]
stroke = topp.Stroke(np.hstack((stroke.t.reshape(-1, 1), newx)),
                     clean=False)
plt.figure(figsize=(4, 3))
plt.plot(*stroke.x.T)
plt.axis('equal')
plt.xlim(0, W)
plt.ylim(0, H);
stroke.retime()

In [ ]:
io.send_spline_serial(stroke.retimed_path.cspl, PORT)

In [ ]:
t, x, v, _ = stroke.sample_retimed(dt=0.1)
print('Ground Truth:')
print(np.hstack((t[:10].reshape(-1, 1), x[:10], v[:10])))
print('Read back from robot:')
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port=PORT) as robot:
    robot.send('t*')
    io.read_spline_serial(robot, num_lines=10)